In [41]:
#!/usr/bin/env python
"""
rmsd_Aloop_backbone.py
——————————————————————————————————————————————
Backbone (N, CA, C, O) RMSD for the Abl1 A‑Loop
(residues 379‑395 in PDB 6XR6) vs an esm3 model.

Files (edit the paths if your layout differs)
  ./data/ground_truth/6xr6.pdb            # numbering 229‑515
  ./data/predicted/esm3_6xr6_229-515.pdb  # numbering 1‑287

Output
  • Global A‑Loop RMSD before + after alignment (printed)
  • Per‑residue RMSD  →  Aloop_per_residue_backbone_rmsd.txt
"""

from pathlib import Path
from collections import defaultdict
import numpy as np
from Bio.PDB import PDBParser, Superimposer

# ------------------------------------------------------------------- #
# 1.  File paths
# ------------------------------------------------------------------- #
ref_path   = Path("./data/ground_truth/6xr6.pdb")
model_path = Path("./data/predicted/esm3_6xr6_229-515.pdb")

# ------------------------------------------------------------------- #
# 2.  Parse the structures (quiet parser = no warnings)
# ------------------------------------------------------------------- #
parser = PDBParser(QUIET=True)
ref_structure   = parser.get_structure("ref",   ref_path)
model_structure = parser.get_structure("model", model_path)

# Use first model / first chain in each file (change if needed)
ref_chain   = next(ref_structure[0].get_chains())
model_chain = next(model_structure[0].get_chains())

# ------------------------------------------------------------------- #
# 3.  A‑Loop residue mapping
# ------------------------------------------------------------------- #
REF_START, REF_END = 379, 395              # numbering in 6XR6
OFFSET             = -228                  # model_resid = ref_resid + OFFSET
MODEL_START, MODEL_END = REF_START + OFFSET, REF_END + OFFSET

BACKBONE = {"N", "CA", "C", "O"}

ref_atoms, model_atoms, resid_lookup = [], [], []

for ref_r, model_r in zip(range(REF_START, REF_END + 1),
                          range(MODEL_START, MODEL_END + 1)):

    if (" ", ref_r, " ") not in ref_chain:
        continue
    if (" ", model_r, " ") not in model_chain:
        continue

    ref_res   = ref_chain[(" ", ref_r, " ")]
    model_res = model_chain[(" ", model_r, " ")]

    for name in BACKBONE:
        if name in ref_res and name in model_res:
            ref_atoms.append(ref_res[name])
            model_atoms.append(model_res[name])
            resid_lookup.append(ref_r)

if not ref_atoms:
    raise RuntimeError("No common backbone atoms found for the A‑Loop!")

print(f"Paired {len(ref_atoms)} backbone atoms "
      f"across {len(set(resid_lookup))} A‑Loop residues")

coords_ref   = np.array([a.get_coord() for a in ref_atoms])
coords_model = np.array([a.get_coord() for a in model_atoms])

# ------------------------------------------------------------------- #
# 4.  RMSD before alignment
# ------------------------------------------------------------------- #
rmsd_pre = np.sqrt(((coords_model - coords_ref) ** 2).sum(axis=1).mean())
print(f"A‑Loop backbone RMSD BEFORE alignment : {rmsd_pre:7.3f} Å")

# ------------------------------------------------------------------- #
# 5.  Optimal superposition (least‑squares fit on the same atoms)
# ------------------------------------------------------------------- #
sup = Superimposer()
sup.set_atoms(ref_atoms, model_atoms)
sup.apply(model_structure.get_atoms())         # rotates *all* model atoms

rmsd_post = sup.rms
print(f"A‑Loop backbone RMSD AFTER  alignment : {rmsd_post:7.3f} Å")

# ------------------------------------------------------------------- #
# 6.  Per‑residue RMSD after alignment
# ------------------------------------------------------------------- #
rot, tran = sup.rotran
coords_model_fit = coords_model @ rot.T + tran
sq_diffs = ((coords_model_fit - coords_ref) ** 2).sum(axis=1)

stats = defaultdict(lambda: [0.0, 0])          # {resid: [sum_sq, count]}
for resid, sq in zip(resid_lookup, sq_diffs):
    stats[resid][0] += sq
    stats[resid][1] += 1

per_res_rmsd = np.array(
    [[resid, np.sqrt(sum_sq / n_atoms)]
     for resid, (sum_sq, n_atoms) in sorted(stats.items())]
)

np.savetxt("Aloop_per_residue_backbone_rmsd.txt",
           per_res_rmsd,
           fmt="%5d  %7.3f",
           header="Resid  RMSD(Å)")
print("Per‑residue RMSD written to Aloop_per_residue_backbone_rmsd.txt")
print("Done.")


Paired 68 backbone atoms across 17 A‑Loop residues
A‑Loop backbone RMSD BEFORE alignment :  15.444 Å
A‑Loop backbone RMSD AFTER  alignment :   6.343 Å
Per‑residue RMSD written to Aloop_per_residue_backbone_rmsd.txt
Done.


In [8]:
from pathlib import Path
from Bio.PDB import MMCIFParser, Superimposer, is_aa, PDBParser

# Paths to your files
ref_path  = Path("./data/ground_truth/6xr6.cif")                # ground‑truth
# ref_path  = Path("./data/ground_truth/6xrg.cif")                # ground‑truth
# pred_path = Path("./data/predicted/esm3_abl1b_complete_229-515.pdb")  # ESM‑3 prediction
# pred_path = Path("./data/predicted/esm3_abl1b_complete_partialdiffusion_0_229-515.pdb")  # ESM‑3 prediction
pred_path = Path("./data/predicted/esm3_abl1b_complete_partialdiffusion_t10_0_229-515.pdb")  # ESM‑3 prediction

def ca_dict(structure, chain_id="A"):
    """
    Return {resseq: CA_atom} for standard amino‑acid residues in *chain_id*.
    Works for PDB or mmCIF parsed with Bio.PDB.
    """
    chain = structure[0][chain_id]          # first model assumed
    return {res.get_id()[1]: res["CA"]
            for res in chain
            if is_aa(res, standard=True) and "CA" in res}

parser = MMCIFParser(QUIET=True)
PDBparser = PDBParser(QUIET=True)
ref_struct  = parser.get_structure("ref" , ref_path)
pred_struct = PDBparser.get_structure("pred", pred_path)

# ---- choose the chains to compare ----
chain_ref  = "A"
chain_pred = "A"            # change if the prediction used a different ID

ref_ca  = ca_dict(ref_struct , chain_ref )
pred_ca = ca_dict(pred_struct, chain_pred)

common_ids = sorted(ref_ca.keys() & pred_ca.keys())
print(f"Common residues: {len(common_ids)}")

if len(common_ids) < 3:
    raise ValueError("Too few overlapping residues to superimpose.")

ref_atoms  = [ref_ca[i]  for i in common_ids]
pred_atoms = [pred_ca[i] for i in common_ids]

sup = Superimposer()
sup.set_atoms(ref_atoms, pred_atoms)

print(f"Backbone Cα RMSD over {len(common_ids)} residues = {sup.rms:.3f} Å")

# Optional: apply the transformation to the prediction object
# sup.apply(pred_struct.get_atoms())

# Optional: save the transformed prediction to inspect in PyMOL / ChimeraX
# from Bio.PDB import PDBIO
# io = PDBIO()
# io.set_structure(pred_struct)
# io.save("esm3_aligned_to_6xr6.pdb")



Common residues: 268


LinAlgError: SVD did not converge